# Boundaries Simplification
#### The boundary files provided by the ONS can be very large.  Large files can sometimes cause memory errors when attempting to display them. To resolve these issues and  display within the Jupyter notbook enviroment it can help to simplify these files.This notebook will show you various techniques to simplify boundaries for display on a folium map . 

#### Where the files are very large or a memory error is encountered when running a simple file level plot, then the simplifyByPoly() method should be used. This method can be speeded up by increasing RAM where possible.

#### For an investigation into the boundary reduction methods and issues see https://metoffice.atlassian.net/wiki/spaces/AD/pages/3736698907/Reduce+Resolution+for+display+of+ONS+Datasets.



## Setup

#### Set up some variables defining the location of your input data.


In [ ]:
SAMPLEDIR = "../cap-sample-data" 
#SAMPLEDIR = "../../../../Datafiles/cap-sample-data/cap-sample-data"


boundary_data = os.path.join(SAMPLEDIR, "shapefiles","NUTS_Level_1_boundries500mgjsn.geojson")
lsoa_data = os.path.join(SAMPLEDIR, "shapefilesFullZip","Lower_Layer_Super_Output_Areas__December_2001__EW_BGC-shp.zip")
naw_regions_data = os.path.join(SAMPLEDIR, "shapefilesFullZip","National_Assembly_for_Wales_Electoral_Regions_(December_2018)_Boundaries_WA_BSC.zip")


#### Open a zipped shapefile and inspect the contents using Geopandas

In [ ]:
import geopandas
import folium

counties_gdf = geopandas.read_file(lsoa_data)
counties_gdf.head()



## Boundary reduction working at file level. The next two code blocks are suitable for displaying smaller files.

#### Reduce boundary file using simplify function in geopandas. This uses shapely's simplify package. 1 unit represents 1 metre. No topology considered.

In [ ]:
# Simplify function in geopandas . This uses shapley's simplify package. 1 unit represents 1 metre. No topology comsidered.simple
import geopandas
import folium
simple_map = folium.Map(location=[50.72039, -1.88092], zoom_start=7)

#Open a shapefile using geopandas
counties_gdf = geopandas.read_file(naw_regions_data)
s_gdf = counties_gdf.simplify(500)
s_gdf.dtype
s_gdf.head

folium.GeoJson(data=counties_gdf["geometry"], style_function=lambda x:{'fillColor': '#228B22', 'color': '#228B22'}, name='originalONSboundaries').add_to(simple_map)

folium.GeoJson(data=s_gdf, style_function=lambda x:{'fillColor': '#00FFFFFF', 'color': '#00FFFFFF'}, name = 'Simplify500_geopandas').add_to(simple_map)

folium.LayerControl().add_to(simple_map)
simple_map

#### Reduce Boundary file using topojson package, preserving topology and tolerance of 500

In [ ]:
#Reduce boundaries using the topojson package, preserving topology and tolerance of 500
boundary_map = folium.Map(location=[50.72039, -1.88092], zoom_start=7)

gdf = counties_gdf
import topojson as tp
topo = tp.Topology(gdf.to_crs({'init':'epsg:3857'}), prequantize=False)# conversion to topo using topojson.
simple = topo.toposimplify(500).to_gdf() #now simplify
simple.crs = ({'init':'epsg:3857'}) #set crs

folium.GeoJson(data=counties_gdf["geometry"], style_function=lambda x:{'fillColor': '#228B22', 'color': '#228B22'},name='originalONSboundaries').add_to(boundary_map)

folium.GeoJson(simple, style_function=lambda x:{'fillColor': '#00FFFFFF', 'color': '#00FFFFFF'}, name = 'topoJsonSimplify500').add_to(boundary_map)

folium.GeoJson(data=s_gdf, style_function=lambda x:{'fillColor': '#FF0000', 'color': '#FF0000'}, name = 'Simplify500_geopandas').add_to(boundary_map)

folium.LayerControl().add_to(boundary_map)

boundary_map

## Boundary reduction working at polygon level . This is suitable for displaying larger or more complex shapefiles 

Reduce Boundary file using Geopandas and Shapleys simplify package. This version loops through polygons to add them iteratively. Use for boundary files with many polygons to add. 

In [ ]:
from folium.folium import Map
# Loop through polygons Iteratively to add to map 
import geopandas as gpd

#Include a filename, a tolerance and a fill colour, and a name for popup column. Returns a folium map object.
def simplifyByPoly(fname, tol, colour, popname):
      df = geopandas.read_file(fname)
      #df = counties_gdf
      m  = folium.Map(location=[50.72039, -1.88092], zoom_start=7)
      df = df.to_crs(epsg=4326) # set to correct projection
      print(df.crs)
      #df.head()

      for _, r in df.iterrows():
          # Without simplifying the representation of each borough,
          # the map might not be displayed
          sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=tol)
          geo_j = sim_geo.to_json()
          geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': colour})
          folium.Popup(r[popname]).add_to(geo_j)
          geo_j.add_to(m) 
      return m



In [ ]:
simplifyByPoly(fname=naw_regions_data, tol=0.001, colour = 'orange', popname = 'nawer18nm')

In [ ]:
simplifyByPoly(fname=lsoa_data, tol=0.001, colour = 'orange', popname = 'LSOA01NM')